In [ ]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import joblib

ruta_train = r"..\data\dataset_train.csv"
df = pd.read_csv(ruta_train)
print(df.head())

X = df.drop("Survived", axis=1).values.astype("float32")
y = df["Survived"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential([
    keras.Input(shape=(X_train.shape[1],)),
    Dense(128, activation="relu"),
    Dense(32, activation="relu"),
    Dropout(0.3),
    Dense(1, activation="sigmoid")
])

model.summary()

model.compile(
    loss="binary_crossentropy",
    optimizer=RMSprop(learning_rate=0.0001),
    metrics=["accuracy"]
)

history = model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=50,
    verbose=2,
    validation_data=(X_test, y_test),
)

# Evaluate
score = model.evaluate(X_test, y_test, verbose=0)

plt.figure(figsize=(12,5))
# ---- LOSS ----
plt.subplot(1,2,1)
plt.plot(history.history["loss"], label="loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
# ---- ACCURACY ----
plt.subplot(1,2,2)
plt.plot(history.history["accuracy"], label="accuracy")
plt.plot(history.history["val_accuracy"], label="val_accuracy")
plt.title("Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.tight_layout()
plt.show()

print("Test loss:", score[0])
print("Test accuracy:", score[1])

model.save("rna_titanic.h5")
joblib.dump(scaler, "rna_scaler.pkl")
print("Modelo: 'rna_titanic.h5' guardado con éxito!")
